In [2]:
import pandas as pd
df=pd.read_excel("Case Study 2 Data.xlsx")
df

,User ID,Budget,Bedrooms,Bathrooms,Qualitative Description
0,1,$500k,3,2,"""I’ve always dreamed of a modern apartment in ..."
1,2,$300k,2,1,"""I’m looking for a cozy little home where I ca..."
2,3,$700k,4,3,"""My dream home is an eco-friendly sanctuary wi..."
3,4,$250k,2,2,"""I need a stylish, open-concept home for my gr..."
4,5,$450k,3,2,"""I want a beachfront property where I can wake..."
5,6,$600k,4,3,"""I’m looking for a luxurious home where I can ..."
6,7,$350k,2,1,"""I’d love a cozy cabin with a fireplace and a ..."
7,8,$400k,3,2,"""I want a historic home with character, close ..."
8,9,$550k,3,2,"""Dreaming of a mountain home with easy access ..."
9,10,$200k,2,1,"""Looking for a small home in a walkable area w..."


In [3]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# Load User Preferences Dataset
user_data = pd.read_csv("user_preferences.csv")
property_data = pd.read_csv("property_data.csv")

# Function to clean and convert budget to numerical values
def clean_budget(budget):
    return int(re.sub(r'[^0-9]', '', budget)) * 1000  # Convert "$500k" → 500000

user_data["Budget"] = user_data["Budget"].apply(clean_budget)
property_data["Price"] = property_data["Price"].apply(clean_budget)

# Normalize numerical features
scaler = MinMaxScaler()
numerical_features = ["Budget", "Bedrooms", "Bathrooms"]
user_data[numerical_features] = scaler.fit_transform(user_data[numerical_features])
property_data[["Price", "Bedrooms", "Bathrooms"]] = scaler.transform(property_data[["Price", "Bedrooms", "Bathrooms"]])

# Load NLP Model (SBERT for sentence embeddings)
model = SentenceTransformer("all-MiniLM-L6-v2")

# Compute embeddings for qualitative descriptions
user_embeddings = model.encode(user_data["Qualitative Description"].fillna(""))
property_embeddings = model.encode(property_data["Description"].fillna(""))

# Compute Similarity Scores
text_similarity = cosine_similarity(user_embeddings, property_embeddings)
numerical_similarity = cosine_similarity(user_data[numerical_features], property_data[["Price", "Bedrooms", "Bathrooms"]])

# Weighted Match Score Calculation
alpha, beta = 0.5, 0.5  # Weight factors for numerical and text similarity
match_scores = alpha * numerical_similarity + beta * text_similarity

# Convert results into a DataFrame
match_df = pd.DataFrame(match_scores, index=user_data["User ID"], columns=property_data["Property ID"])

# Save results
match_df.to_csv("match_scores.csv")
print("Match Score calculation completed and saved to match_scores.csv")

ModuleNotFoundError: No module named 'sentence_transformers'